In [1]:
# Sign-in
from pydrive.auth import GoogleAuth

gauth = GoogleAuth()
gauth.LocalWebserverAuth() # Creates local webserver and auto handles authentication.

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?client_id=839676677362-7ile7rmhde5089d1dgm6akpel294it4k.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&access_type=offline&response_type=code

Authentication successful.


In [2]:
# - - - - - - Import necessary libraries - - - - - -

# Import pydrive library
from pydrive.drive import GoogleDrive

# Import datatime to work with time series
import datetime
from datetime import date

# Import os to work with local files and folders
from os import listdir
from os.path import isfile, join

# Import smtplib to work with mail server
import smtplib, ssl

In [3]:
def GetAllFolders() :
    drive = GoogleDrive(gauth)
    folders = drive.ListFile({'q': "mimeType = 'application/vnd.google-apps.folder' and trashed = false"}).GetList()
    return folders

In [4]:
def GetFilesWithExt(ext) :
    files = [file for file in listdir(mypath) if isfile(join(mypath, f))]
    extFiles = []
    for file in files :
        if file[-3 : ] == ext :
            extFiles.append(file)
    return extFiles

In [5]:
def ShowAllFolders(folders) :
    print('Here are all the folders of your drive')
    getMaxTitleLen = 0
    getMaxIdLen = 0
    for folder in folders :
        getMaxTitleLen = max(getMaxTitleLen, len(folder['title']))
        getMaxIdLen = max(getMaxIdLen, len(folder['id']))
    # Print folders table
    numColLen = len('No.') + len(' ' * 6)
    titColLen = getMaxTitleLen + 6
    idColLen = getMaxIdLen + 6
    print('No.' + ' ' * (numColLen - len('No.')) + '|TITLE' + ' ' * (titColLen - len('TITLE')) + '|ID')
    print('=' * (numColLen + titColLen + idColLen))
    No = 1;
    for folder in folders :
        print(str(No) + ' ' * (numColLen - len(str(No))) + '|' + folder['title'] + ' ' * (titColLen - len(folder['title'])) + '|' + folder['id'])
        print('-' * (numColLen + titColLen + idColLen))
        No += 1

In [6]:
def CreateANewFolder(parent_name, folder_name) :
    drive = GoogleDrive(gauth)
    folders = GetAllFolders()
    parent_id = ""
    for folder in folders :
        if folder['title'] == parent_name :
            parent_id = folder['id']
            break
    if parent_name == 'root' :
        newFolder = drive.CreateFile({
            'title': '[' + str(date.today()) + '] ' + folder_name,
            'mimeType': 'application/vnd.google-apps.folder'
        })
    else :
        newFolder = drive.CreateFile({
            'title': '[' + str(date.today()) + '] ' + folder_name,
            'parents': [{'id': parent_id}],
            'mimeType': 'application/vnd.google-apps.folder'
        })
    newFolder.Upload()
    return newFolder

In [7]:
def WhereToUpload(dest) :
    folderId = dest
    if dest.lower() != 'root' :
        folderId = dest
    return folderId

In [8]:
def UploadFiles(filesContainer, folderId) :
    drive = GoogleDrive(gauth)
    numFiles = 0
    detailInfo = ""
    dest = WhereToUpload(folderId)
    for file in listdir(filesContainer) :
        if dest.lower() != 'root' :
            uploadFile = drive.CreateFile({
                'parents' : [{'id' : dest}],
                'title': file[ : -4]
            })
        else :
            uploadFile = drive.CreateFile({
                'title': file[ : -4]
            })
        uploadFile.SetContentFile(filesContainer + '\\' + file)
        uploadFile.Upload()
        numFiles += 1
        detailInfo += """
        %d. %s
            Size: %s
            Modified Date: %s
        - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
        """ % (numFiles, uploadFile['title'], str(round(int(uploadFile['fileSize'])/1024, 2)) + ' KB', uploadFile['modifiedDate'][ : 10])
    print('\n' + str(numFiles) + ' file(s) successfully uploaded')
    return (numFiles, detailInfo)

In [11]:
# ===============================================
# = = = = = = = = = MAIN DRIVER = = = = = = = = =
# ===============================================

import getpass    # https://www.geeksforgeeks.org/getpass-and-getuser-in-python-password-without-echo/

# Get all user's drive folders
folders = GetAllFolders()

# Display them under a table
ShowAllFolders(folders)

# Ask user for his/her option
folderId = ""
option = input('Do you want to upload files to existing folders or create a new one?\n1. Upload to existing folders\n2. Create a new folder\n')
if(int(option) == 1) :
    # Ask user for their desire drive folder to upload files
    dest = input('Please specify the folder id you want to upload files: > ')
    folderId = WhereToUpload(dest)
else :
    parent_name = input('Please enter a parent folder name: ')
    folder_name = input('Please enter name of a new folder: ')
    newFolder = CreateANewFolder(parent_name, folder_name)
    folderId = newFolder['id']
    

# Ask user for their files container on local PC
filesContainer = input('Please enter the files path: > ')    # C:\\Users\\dacluc\\Desktop\\Upload_Files

# Ask user for their email address to inform them when the uploading process finished
# receiverEmail = getpass.getpass(prompt = "Please enter your email address, we'll inform you when uploading completed: > ")
receiverEmail = input("Please enter your email address, we'll inform you when uploading completed: > ")

# Files uploading starts from now . . .
numFiles, detailInfo = UploadFiles(filesContainer, folderId)
# print(detailInfo)

# After uploaded, send user an email to kindly inform them
# SendMailUsingSMTP(receiverEmail, "Hello world")
# SendMailUsingSMTP2(receiverEmail.rstrip(), detailInfo)

# - - - - - - SEND MAIL TO USER - - - - - -

port = 587  # For starttls
smtp_server = "smtp.gmail.com"
sender_email = "YOUREMAIL"    # Your email address
receiver_email = receiverEmail
# password = input("Type your password and press enter:")
password = 'YOURPASSWORD'    # Your password
message = """\
Subject: Noti from Google Drive

Hooray! %d files(s) have been successfully uploaded
- - - - - - - - -

%s
""" % (numFiles, detailInfo)
context = ssl.create_default_context()
with smtplib.SMTP(smtp_server, port) as server:
#     server.ehlo()  # Can be omitted
    server.starttls(context=context)
#     server.ehlo()  # Can be omitted
    server.login(sender_email, password)
    server.sendmail(sender_email, receiver_email, message)

Here are all the folders of your drive
No.      |TITLE           |ID
1        |Sub-TEMP 3      |1EyrMN56tAsiuWEGgnyacJGrorN8CYeB9
----------------------------------------------------------------
2        |Sub-TEMP 2      |1LupMq7HtOayYqBtCwOzse5BN8QWQDHbD
----------------------------------------------------------------
3        |Sub-TEMP 1      |1LenoQ_mO5O-QVq-GP1tVe9gQfL9cHcoX
----------------------------------------------------------------
4        |TEMP 3          |1xFk3hA9YV-Ufcxk7MFql_ta78I_0y_o7
----------------------------------------------------------------
5        |TEMP 2          |1aY-vvhO6jbtmr4FzP3tk3pp5VUi3qD_P
----------------------------------------------------------------
6        |TEMP 1          |1huQFf9t2d5e4OrD2tmIVnIWtAVdbuEVV
----------------------------------------------------------------
Do you want to upload files to existing folders or create a new one?
1. Upload to existing folders
2. Create a new folder
2
Please enter a parent folder name: root
Please ente